In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import ols
import re
import scipy.stats as stats
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from itertools import combinations
from sklearn import neighbors
from sklearn.impute import KNNImputer
# plt.style.use('ggplot')

In [2]:
def read_csvs():
    df_hanes = pd.read_csv('nhanes.csv')
    df_hanes.columns = [col.lower() for col in df_hanes.columns]
    return df_hanes

def create_predictors(source_df, predictors):
    """
        Each predictor is of the form: ('unemployed', ''bst90p23_y')
    """
    df = pd.DataFrame([])
    for pred in predictors:
        df[pred[0]] = source_df[pred[1]]
    return df

def create_dummies(source_df, pred_df, dummy_tuples):
    for dummy_tup in dummy_tuples:
        dummy = pd.get_dummies(source_df[dummy_tup[1]].astype('category'), prefix=dummy_tup[0],drop_first=True)
        pred_df = pd.concat([pred_df, dummy], axis=1)
    return pred_df

def knn_impute(df, col_list_to_impute):
    imputer = KNNImputer(n_neighbors=15)
    df_filled = imputer.fit_transform(df)
    new_df = pd.DataFrame(df_filled)
    new_df.columns = df.columns
    for c in col_list_to_impute:
        df[c] = new_df[c]
    return df

dummies = [
           ('gender', 'riagendr'),
           ('race', 'ridreth3')
          ]

predictors = [
              ('chol', 'lbxtc'),
              ('trig', 'lbxstr'),
              ('age', 'ridageyr'),
              ('bmi', 'bmxbmi'),
              ('bp', 'bpxsy2'),
              ('bun', 'lbxsbu'),
             ]

df_hanes = read_csvs()

# Age
df_hanes['ridageyr'] = df_hanes['ridageyr'].apply(lambda x: np.nan if (x < 1) | (x >= 80) else x)

pred_df = create_predictors(df_hanes, predictors)
pred_df = create_dummies(df_hanes, pred_df, dummies)
# pred_df = knn_impute(pred_df, ['chol','trig','bmi','bp','age','bun','gender_2','race_2','race_3','race_4','race_6','race_7'])


pred_df.columns = [col.replace('.','_') for col in pred_df.columns]

/Users/rebjl/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (379,380,396) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [9]:
print(pred_df['chol'].isna().sum())
print(pred_df['trig'].isna().sum())
print(pred_df['bmi'].isna().sum())

8034
11130
3489


In [20]:
import random

random.seed(0)
r = random.random()

def num_nas(series):   
    print(f"{series.name} na values {series.isna().sum()}")

num_nas(pred_df['chol'])
num_nas(pred_df['trig'])

pred_df['chol_rand'] = pred_df['chol'].map(
    lambda x: random.random() if x not in [np.nan] else 1)

pred_df['trig_rand'] = pred_df['trig'].map(
    lambda x: random.random() if x not in [np.nan] else 1)

num_nas(pred_df['chol_rand'])
num_nas(pred_df['trig_rand'])

corr = pred_df['chol'].corr(pred_df['trig'])
print(f"correlation between chol_rand & trig_rand: {corr}")


chol na values 8034
trig na values 11130
chol_rand na values 0
trig_rand na values 0
correlation between chol_rand & trig_rand: 0.37807054287590175


## The impact of biomedical measures on total cholesterol levels in young, middle-aged, and elderly populations in the US 2013-2016.
DV: Total Cholesterol 
IV: Triglycerides, BMI, Blood Pressure, BUN, Age, Gender, Race

### Plots of Non-Transformed, Non-Scaled

In [ ]:
# Correlation and Histogram
sns.jointplot(pred_df['chol'], pred_df['trig'], kind='reg')

In [ ]:
# Continuous Vars
pred_df.bmi.plot(kind='hist', alpha=0.5)
pred_df.bp.plot(kind='hist', alpha=0.5)
pred_df.bun.plot(kind='hist', alpha=0.5)
pred_df.chol.plot(kind='hist', alpha=0.5)
pred_df.trig.plot(kind='hist', alpha=0.5)

plt.legend()

In [ ]:
# Correlation Matrix / Heatmap

corr = pred_df.corr()
ax = sns.heatmap(
    corr, 
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    square=True
)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right'
);

In [ ]:
pred_df = pred_df.dropna()
print(f'Length of dataframe after drop NAs: {len(pred_df)}')

X = pred_df.drop(['chol'], axis=1)
y = pred_df['chol']
print('Ready for Regression')

In [ ]:
regression = LinearRegression()
crossvalidation = KFold(n_splits=3, shuffle=True, random_state=1)

baseline = np.mean(cross_val_score(regression, X, y, scoring='r2', cv=crossvalidation))
print(f'R-squared score for non-normal model: {baseline}')

### Assumptions

In [ ]:
X.columns

In [ ]:
model =ols('chol~trig+age+bmi+bp+bun+gender_2+race_2+race_2+race_3+race_4+race_6+race_7',data=pred_df).fit()
model.summary()

In [ ]:

plt.rc('figure', figsize=(11, 8.5))
#plt.text(0.01, 0.05, str(model.summary()), {'fontsize': 12}) old approach
plt.text(0.01, 0.05, str(model.summary()), {'fontsize': 10}, fontproperties = 'monospace') # approach improved by OP -> monospace!
plt.axis('off')
plt.tight_layout()
plt.savefig('model_summary.png')

In [ ]:
pred_val = model.fittedvalues.copy()
true_val = pred_df['chol'].values.copy()
residual = true_val - pred_val

In [ ]:
fig, ax = plt.subplots(figsize=(6,2.5))
ax.scatter(pred_df['trig'],residual)

In [ ]:
# checking for normality - QQ plot 
import statsmodels.api as sm
import scipy as sp

fig, ax = plt.subplots(figsize=(6,2.5))
sp.stats.probplot(residual, plot=ax, fit=True)
residual**2

sm.graphics.qqplot(residual, dist=sp.stats.norm) # me
0.9523990893322951

In [ ]:
sns.distplot(residual)

### Plots of Log-Transformed

In [ ]:
def read_csvs():
    df_hanes = pd.read_csv('nhanes.csv')
    df_hanes.columns = [col.lower() for col in df_hanes.columns]
    return df_hanes

def create_predictors(source_df, predictors):
    """
        Each predictor is of the form: ('unemployed', ''bst90p23_y')
    """
    df = pd.DataFrame([])
    for pred in predictors:
        df[pred[0]] = source_df[pred[1]]
    return df

def create_dummies(source_df, pred_df, dummy_tuples):
    for dummy_tup in dummy_tuples:
        dummy = pd.get_dummies(source_df[dummy_tup[1]].astype('category'), prefix=dummy_tup[0],drop_first=True)
        pred_df = pd.concat([pred_df, dummy], axis=1)
    return pred_df

def knn_impute(df, col_list_to_impute):
    imputer = KNNImputer(n_neighbors=15)
    df_filled = imputer.fit_transform(df)
    new_df = pd.DataFrame(df_filled)
    new_df.columns = df.columns
    for c in col_list_to_impute:
        df[c] = new_df[c]
    return df

dummies = [
           ('gender', 'riagendr'),
           ('race', 'ridreth3')
          ]

predictors = [
              ('chol', 'lbxtc'),
              ('trig', 'lbxstr'),
              ('age', 'ridageyr'),
              ('bmi', 'bmxbmi'),
              ('bp', 'bpxsy2'),
              ('bun', 'lbxsbu'),
             ]

df_hanes = read_csvs()

# Log Transformation
non_normal = [
              'lbxgh', 
              'lbxtc',
              'lbxstr',
              'bmxbmi', 
              'bpxsy2',
              'lbxsbu'
             ]

for feat in non_normal:
    df_hanes[feat] = df_hanes[feat].map(lambda x: np.log(x))

# Normalization - Finish writing loop
normalize = []
for feat in normalize:
    df_hanes[feat] = df_hanes[feat]
    

# Age
df_hanes['ridageyr'] = df_hanes['ridageyr'].apply(lambda x: np.nan if (x < 1) | (x >= 80) else x)

pred_df = create_predictors(df_hanes, predictors)
pred_df = create_dummies(df_hanes, pred_df, dummies)
pred_df = knn_impute(pred_df, ['chol','trig','bmi','bp','age','bun','gender_2','race_2','race_3','race_4','race_6','race_7'])


pred_df.columns = [col.replace('.','_') for col in pred_df.columns]

In [ ]:
# Correlation and Histogram
sns.jointplot(pred_df['chol'], pred_df['trig'], kind='reg')

In [ ]:
# Continuous Vars
pred_df.bmi.plot(kind='hist', alpha=0.5)
pred_df.bp.plot(kind='hist', alpha=0.5)
pred_df.bun.plot(kind='hist', alpha=0.5)
pred_df.chol.plot(kind='hist', alpha=0.5)
pred_df.trig.plot(kind='hist', alpha=0.5)

plt.legend()

In [ ]:
# Correlation Matrix / Heatmap

corr = pred_df.corr()
ax = sns.heatmap(
    corr, 
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    square=True
)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right'
);

### Min-Max Scaled Log-Transformed

In [ ]:
bmi = pred_df.bmi
bp = pred_df.bp
bun = pred_df.bun
chol = pred_df.chol
trig = pred_df.trig

# min-max
scaled_bmi = (bmi - min(bmi)) / (max(bmi) - min(bmi))
scaled_bp = (bp - min(bp)) / (max(bp) - min(bp))
scaled_bun = (bun - min(bun)) / (max(bun) - min(bun))
scaled_chol = (chol - min(chol)) / (max(chol) - min(chol))
scaled_trig = (trig - min(trig)) / (max(trig) - min(trig))


data_cont_scaled = pd.DataFrame([])
data_cont_scaled['bmi'] = scaled_bmi
data_cont_scaled['bp'] = scaled_bp
data_cont_scaled['bun'] = scaled_bun
data_cont_scaled['chol'] = scaled_chol
data_cont_scaled['trig'] = scaled_trig


data_cont_scaled.hist(figsize = [10, 6]);

In [ ]:
pred_df['bmi'] = data_cont_scaled['bmi']
pred_df['bp'] = data_cont_scaled['bp']
pred_df['bun'] = data_cont_scaled['bun']
pred_df['chol'] = data_cont_scaled['chol']
pred_df['trig'] = data_cont_scaled['trig']
print('Inputted scaled data back into main dataframe')

In [ ]:
pred_df = pred_df.dropna()
print(f'Length of dataframe after drop NAs: {len(pred_df)}')

X = pred_df.drop(['chol'], axis=1)
y = pred_df['chol']
print('Ready for Regression')

In [ ]:
regression = LinearRegression()
crossvalidation = KFold(n_splits=3, shuffle=True, random_state=1)

baseline = np.mean(cross_val_score(regression, X, y, scoring='r2', cv=crossvalidation))
baseline

In [ ]:
# 0.233
# 0.2156 w/ imputation k=5
# 0.236 w/ imputation k=10
# 0.238 w/ imputation k=15

### Interactions - Age and BUN

In [ ]:
pred_df['age'] = pred_df['age'].apply(lambda x: 1 if x < 40 else x)
pred_df['age'] = pred_df['age'].apply(lambda x: 2 if (x >= 40) & (x < 60) else x)
pred_df['age'] = pred_df['age'].apply(lambda x: 3 if (x >= 60) & (x < 80) else x)

age_1 = pred_df[pred_df['age'] == 1]
age_2 = pred_df[pred_df['age'] == 2]
age_3 = pred_df[pred_df['age'] == 3]
print('Grouped Age into 3 Groups')

In [ ]:
pred_df.boxplot('chol',by = 'age',figsize = (7,7))

In [ ]:
regression_1 = LinearRegression()
regression_2 = LinearRegression()
regression_3 = LinearRegression()

bun_1 = age_1['bun'].values.reshape(-1, 1)
bun_2 = age_2['bun'].values.reshape(-1, 1)
bun_3 = age_3['bun'].values.reshape(-1, 1)

regression_1.fit(bun_1, age_1['chol'])
regression_2.fit(bun_2, age_2['chol'])
regression_3.fit(bun_3, age_3['chol'])


# Make predictions using the testing set
pred_1 = regression_1.predict(bun_1)
pred_2 = regression_2.predict(bun_2)
pred_3 = regression_3.predict(bun_3)


# The coefficients
print(f'Regression Coefficient 1: {regression_1.coef_}')
print(f'Regression Coefficient 2: {regression_2.coef_}')
print(f'Regression Coefficient 3: {regression_3.coef_}')


In [ ]:
plt.figure(figsize=(10,6))

plt.scatter(bun_1, age_1['chol'],  color='blue', alpha=0.3, label = 'age = 1')
plt.scatter(bun_2, age_2['chol'],  color='red', alpha=0.3, label = 'age = 2')
plt.scatter(bun_3, age_3['chol'],  color='white', alpha=0.3, label = 'age = 3')

plt.plot(bun_1, pred_1, color='blue', linewidth=2)
plt.plot(bun_2, pred_2, color='red', linewidth=2)
plt.plot(bun_3, pred_3, color='green', linewidth=2)

plt.ylabel('chol')
plt.xlabel('bun')
plt.legend();

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(11, 8.5))
fig.suptitle("Predicting Cholesterol using BUN with age group as predictor", fontsize=16)
ax.set_title("K-Nearest Neighbors Interpolation=15")
# plot_ax.legend([home.split(' ', 1)[0], away.split(' ', 1)[0]], loc='upper right')
ax.scatter(bun_1, age_1['chol'],  color='blue', alpha=0.3, label = 'age < 40')
ax.scatter(bun_2, age_2['chol'],  color='red', alpha=0.3, label = '40 < age < 60')
ax.scatter(bun_3, age_3['chol'],  color='white', alpha=0.3, label = '60 < age < 80')
ax.plot(bun_1, pred_1, color='blue', linewidth=2)
ax.plot(bun_2, pred_2, color='red', linewidth=2)
ax.plot(bun_3, pred_3, color='green', linewidth=2)
ax.set_ylabel('Cholesterol mg/dL (chol)')
ax.set_xlabel('Blood Urea Nitrogen mg/dL (bun)')
# ax.ylabel('Cholesterol mg/dL (chol)')
# ax.xlabel('Blood Urea Nitrogen mg/dL (bun)')
ax.legend();
plt.savefig("predicting cholesterol.png")

In [ ]:
regression = LinearRegression()
crossvalidation = KFold(n_splits=3, shuffle=True, random_state=1)

X_interact = X.copy()
X_interact['bun_age'] = X['bun'] * X['age']

interact_bun_age = np.mean(cross_val_score(regression, X_interact, y, scoring='r2', cv=crossvalidation))
print(f'Regression Coefficient for Interaction: {interact_bun_age}')

## 0.25!

### Check for statistical Significance

In [ ]:
import statsmodels.api as sm
X_interact = sm.add_constant(X_interact)
model = sm.OLS(y,X_interact)
results = model.fit()

results.summary()